In [46]:
import os 
import pandas as pd 
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import BorderlineSMOTE
import dask.dataframe as dd
import fastparquet
from sklearn.svm import OneClassSVM
from sklearn.feature_selection import VarianceThreshold

try:
    os.chdir("C:/Users/Sam/Documents/SISE/Fouille de données")
except:
    os.chdir("/Users/titouanhoude/Documents/GitHub")
    
train = pd.read_parquet('train.parquet.gzip')
test = pd.read_parquet('test.parquet.gzip')

Xtrain = train.drop(["FlagImpaye", "ZIBZIN", "Date", "Heure_split", "DateTransaction", "CodeDecision", "Unnamed: 0"], axis = 1)

Ytrain = pd.DataFrame(train.FlagImpaye)
Ytrain = train['FlagImpaye'].astype('int')

Xtest  = test.drop(["FlagImpaye","ZIBZIN", "Date", "Heure_split", "DateTransaction","CodeDecision", 'Unnamed: 0'], axis = 1)
Ytest  = test.FlagImpaye

In [3]:
from sklearn.utils import class_weight

classes = np.unique(Ytrain)
cw = class_weight.compute_class_weight(class_weight = 'balanced',classes =  np.unique(Ytrain),y= Ytrain)
weights = dict(zip(classes,cw))

In [ ]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(class_weight = weights, n_estimators=1000,random_state=0).fit(Xtrain, Ytrain)
pred = lgbm.predict(Xtest_norm)
   
ctest = confusion_matrix(Ytest, pred)
ftest = (2*ctest[1,1]/(2*ctest[1,1]+ctest[0,1]+ctest[1,0])) * 100
ftest

In [ ]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(class_weight = weights,max_depth=50, alpha=0.7, loss='deviance', learning_rate=0.09, n_estimators=1000, subsample=0.6,reg_alpha = 1.6, reg_lambda = 1.1,random_state=0).fit(Xtrain, Ytrain)
pred = lgbm.predict(Xtest_norm)
   
ctest = confusion_matrix(Ytest, pred)
ftest = (2*ctest[1,1]/(2*ctest[1,1]+ctest[0,1]+ctest[1,0])) * 100
ftest

In [64]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()
Xtrain_norm = normalizer.transform(Xtrain)
Xtest_norm = normalizer.transform(Xtest)

C:\Users\Sam\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but Normalizer was fitted without feature names
  warnings.warn(
C:\Users\Sam\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but Normalizer was fitted without feature names
  warnings.warn(


In [65]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(class_weight = weights,max_depth=50, alpha=0.7, loss='deviance', learning_rate=0.09, n_estimators=1000, subsample=0.6,reg_alpha = 1.6, reg_lambda = 1.1,random_state=0).fit(Xtrain_norm, Ytrain)
pred = lgbm.predict(Xtest_norm)
   
ctest = confusion_matrix(Ytest, pred)
ftest = (2*ctest[1,1]/(2*ctest[1,1]+ctest[0,1]+ctest[1,0])) * 100
ftest

4.769882455495171

In [63]:
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier( LGBMClassifier(class_weight = weights,max_depth=50, alpha=0.7, loss='deviance', learning_rate=0.09, n_estimators=1000, subsample=0.6,reg_alpha = 1.6, reg_lambda = 1.1,random_state=0),n_estimators=150, random_state=0).fit(Xtrain, Ytrain)

pred = clf.predict(Xtest)
   
ctest = confusion_matrix(Ytest, pred)
ftest = (2*ctest[1,1]/(2*ctest[1,1]+ctest[0,1]+ctest[1,0])) * 100
ftest

KeyboardInterrupt: 

In [47]:
from sklearn.feature_selection import SelectFromModel

model = SelectFromModel(lgbm, prefit=True)
Xtrain_2 = Xtrain[Xtrain.columns[model.get_support(indices = True)]]
Xtest_2 = Xtest[Xtest.columns[model.get_support(indices = True)]]

In [48]:

lgbm2 = LGBMClassifier(class_weight = weights,max_depth=50, alpha=0.7, loss='deviance', learning_rate=0.09, n_estimators=1000, subsample=0.6,reg_alpha = 1.6, reg_lambda = 1.1,random_state=0).fit(Xtrain_2, Ytrain)
pred2 = lgbm2.predict(Xtest_2)
   
ctest2 = confusion_matrix(Ytest, pred2)
ftest2 = (2*ctest2[1,1]/(2*ctest2[1,1]+ctest2[0,1]+ctest2[1,0])) * 100
ftest2

5.170864558920251